In [ ]:
import time
from itertools import product

# Field setup
q = 2**61 - 1
Fq = GF(q)

# Polynomial ring with 9 variables
R = PolynomialRing(Fq, ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9'])
X1, X2, X3, X4, X5, X6, X7, X8, X9 = R.gens()

def generate_dense_multilinear_polynomial(R, num_vars=9):
    """
    Generate a dense multilinear polynomial with all 2^num_vars terms.
    Each term is a product of distinct variables or their complements.
    """
    variables = R.gens()
    poly = R.zero()
    
    # Generate all 2^num_vars combinations
    for combination in product([0, 1], repeat=num_vars):
        # Random coefficient
        coeff = Fq.random_element()
        
        # Build the term
        term = coeff
        for i, bit in enumerate(combination):
            if bit == 1:
                term *= variables[i]
            else:
                term *= (1 - variables[i])
        
        poly += term
    
    return poly

def time_polynomial_product(N, num_vars=9):
    """
    Generate N dense multilinear polynomials and time their product.
    """
    print(f"Generating {N} dense multilinear polynomials with {num_vars} variables...")
    
    # Generate polynomials
    start_gen = time.time()
    polynomials = []
    for i in range(N):
        poly = generate_dense_multilinear_polynomial(R, num_vars)
        polynomials.append(poly)
        if (i + 1) % max(1, N // 10) == 0:
            print(f"Generated {i + 1}/{N} polynomials")
    
    gen_time = time.time() - start_gen
    print(f"Generation time: {gen_time:.3f} seconds")
    
    # Compute product
    print(f"Computing product of {N} polynomials...")
    start_prod = time.time()
    
    product_poly = polynomials[0]
    for i in range(1, N):
        product_poly = product_poly * polynomials[i]
        if (i + 1) % max(1, N // 10) == 0:
            print(f"Multiplied {i + 1}/{N} polynomials")
    
    prod_time = time.time() - start_prod
    total_time = gen_time + prod_time
    
    print(f"\nResults for N = {N}:")
    print(f"Generation time: {gen_time:.3f} seconds")
    print(f"Product computation time: {prod_time:.3f} seconds")
    print(f"Total time: {total_time:.3f} seconds")
    print(f"Number of terms in product: {len(product_poly.monomials())}")
    print(f"Total degree of product: {product_poly.total_degree()}")
    
    return product_poly, gen_time, prod_time, total_time

# Test with different values of N
test_values = [2, 3, 4, 5]  # Start small due to exponential growth

print("Field: GF(2^61 - 1)")
print("Variables: X1, X2, X3, X4, X5, X6, X7, X8, X9")
print("Each polynomial has 2^9 = 512 terms\n")

results = {}
for N in test_values:
    try:
        product_poly, gen_time, prod_time, total_time = time_polynomial_product(N)
        results[N] = {
            'gen_time': gen_time,
            'prod_time': prod_time,
            'total_time': total_time,
            'num_terms': len(product_poly.monomials()),
            'total_degree': product_poly.total_degree()
        }
        print("-" * 60)
    except MemoryError:
        print(f"Memory error for N = {N}")
        break
    except KeyboardInterrupt:
        print(f"Interrupted for N = {N}")
        break

# Summary
print("\nSUMMARY:")
print("N\tGen Time\tProd Time\tTotal Time\tTerms\tDegree")
print("-" * 70)
for N, data in results.items():
    print(f"{N}\t{data['gen_time']:.3f}s\t\t{data['prod_time']:.3f}s\t\t{data['total_time']:.3f}s\t\t{data['num_terms']}\t{data['total_degree']}")

Field: GF(2^61 - 1)
Variables: X1, X2, X3, X4, X5, X6, X7, X8, X9
Each polynomial has 2^9 = 512 terms

Generating 2 dense multilinear polynomials with 9 variables...
Generated 1/2 polynomials
Generated 2/2 polynomials
Generation time: 0.257 seconds
Computing product of 2 polynomials...
Multiplied 2/2 polynomials

Results for N = 2:
Generation time: 0.257 seconds
Product computation time: 0.263 seconds
Total time: 0.520 seconds
Number of terms in product: 19683
Total degree of product: 18
------------------------------------------------------------
Generating 3 dense multilinear polynomials with 9 variables...
Generated 1/3 polynomials
Generated 2/3 polynomials
Generated 3/3 polynomials
Generation time: 0.430 seconds
Computing product of 3 polynomials...
Multiplied 2/3 polynomials
Multiplied 3/3 polynomials

Results for N = 3:
Generation time: 0.430 seconds
Product computation time: 12.298 seconds
Total time: 12.728 seconds
Number of terms in product: 262144
Total degree of product: 27


In [5]:
import time
from itertools import product
from math import prod

# ===== Field and ring (yours) =====
q  = 2**61 - 1
Fq = GF(q)
R  = PolynomialRing(Fq, ['X1','X2','X3','X4','X5','X6','X7','X8','X9'])
X  = R.gens()
n  = len(X)

# ===== Utilities: index math for n-D tensors =====
def strides_from_dims(dims):
    # C-order: last axis is contiguous
    s = [1]*len(dims)
    for i in range(len(dims)-2, -1, -1):
        s[i] = s[i+1]*dims[i+1]
    return s

def idx_from_multi(k, strides):
    return sum(k[i]*strides[i] for i in range(len(k)))

def iter_multi(dims):
    # Cartesian product over ranges(dims[i]) in axis order 0..n-1
    return product(*[range(d) for d in dims])

# ===== Convert multilinear polynomial <-> tensor on {0,1}^n =====
def poly_to_tensor2(P, num_vars=n):
    """
    Return a flat list of length 2^n with coefficients c[e],
    where e_i in {0,1} and monomial is X1^e1 * ... * Xn^en.
    """
    dims = [2]*num_vars
    size = 1 << num_vars
    out  = [Fq(0)]*size
    strides = strides_from_dims(dims)

    # P.dict(): keys are ETuple exponent vectors, values are coeffs
    for etu, a in P.dict().items():
        e = tuple(etu)                      # <-- ETuple -> Python tuple of exponents
        # sanity: multilinear input only
        if any(ee > 1 for ee in e):
            raise ValueError("Not multilinear: exponent > 1 in some variable.")
        idx = idx_from_multi(e, strides)
        out[idx] += a
    return out, dims

def tensor2_to_poly(coeffs, dims, ring=R, vars_=X):
    """Build R-polynomial sum_{k} coeffs[k] * Π Xi^{k_i} for k_i < dims[i]."""
    strides = strides_from_dims(dims)
    P = ring.zero()
    for k in iter_multi(dims):
        c = coeffs[idx_from_multi(k, strides)]
        if c != 0:
            m = ring.one()
            for i, ki in enumerate(k):
                if ki:
                    m *= vars_[i]**ki
            P += c * m
    return P

# ===== Tiny DFT matrices over Fq for a given length L =====
def pick_length_L(N, p_minus_1=q-1, candidates=(3,5,7,9,11,13,15,17,19,21,27,25)):
    """Smallest odd L >= N+1 with L | (p-1)."""
    for L in candidates:
        if L >= N+1 and (p_minus_1 % L) == 0:
            return L
    raise ValueError("No suitable small L found; extend candidates.")

def primitive_Lth_root(L):
    g = Fq.multiplicative_generator()
    return g^((q-1)//L)

def precompute_dft_tables(L, w):
    """
    Return (F, Finv) where F[i][j] = w^(i*j), Finv[i][j] = (1/L)*w^(-i*j).
    Stored as lists of lists of Fq elements for speed in Python loops.
    """
    invL = Fq(1) / Fq(L)
    pow_cache = [[w**(i*j) for j in range(L)] for i in range(L)]
    F     = pow_cache
    winv  = w**(-1)
    pow_cache_inv = [[winv**(i*j) for j in range(L)] for i in range(L)]
    Finv  = [[invL*pow_cache_inv[i][j] for j in range(L)] for i in range(L)]
    return F, Finv

# ===== In-place n-D DFT along each axis (separable) =====
def transform_along_axis(flat, dims, axis, Fmat):
    """
    In-place multiply each length-L line along `axis` by the LxL matrix Fmat.
    flat: list length prod(dims)
    """
    L = dims[axis]
    strides = strides_from_dims(dims)
    s_axis  = strides[axis]
    # Number of independent lines = total_size / (L * s_axis)
    total   = prod(dims)
    lines   = total // (L * s_axis)
    # For each line, we have s_axis interleaved scalars per position
    # We process each of the s_axis lanes independently.
    base_stride_block = L * s_axis
    ptr = 0  # we'll index via computed bases; keeping ptr not used
    for line in range(lines):
        base0 = line * base_stride_block
        for lane in range(s_axis):
            # Gather x[0..L-1]
            x = [flat[base0 + t*s_axis + lane] for t in range(L)]
            # y = Fmat * x
            for i in range(L):
                acc = Fq(0)
                Fi = Fmat[i]
                # Since L is tiny (3 or 5 typically), do the naive dot
                for j in range(L):
                    acc += Fi[j] * x[j]
                flat[base0 + i*s_axis + lane] = acc

def nd_dft_inplace(flat, dims, Fmat):
    for axis in range(len(dims)):
        transform_along_axis(flat, dims, axis, Fmat)

def nd_idft_inplace(flat, dims, Finv):
    for axis in range(len(dims)):
        transform_along_axis(flat, dims, axis, Finv)

# ===== Pad from shape (2,...,2) to (L,...,L) and back =====
def pad_tensor(coeffs2, dims2, L):
    """
    coeffs2: flat list over dims2 = [2]*n
    Return flat list over dimsL = [L]*n with zeros elsewhere.
    """
    n = len(dims2)
    dimsL = [L]*n
    totalL = L**n
    out = [Fq(0)] * totalL
    strides2 = strides_from_dims(dims2)
    stridesL = strides_from_dims(dimsL)
    for e in iter_multi(dims2):
        c = coeffs2[idx_from_multi(e, strides2)]
        out[idx_from_multi(e, stridesL)] = c  # same index where e_i ∈ {0,1}
    return out, dimsL

def crop_tensor(flatL, dimsL, upto):
    """
    Extract the first upto entries along each axis (upto <= dimsL[i]) into
    a new flat tensor with dimsU = [upto]*n.
    """
    n = len(dimsL)
    dimsU = [upto]*n
    stridesL = strides_from_dims(dimsL)
    stridesU = strides_from_dims(dimsU)
    out = [Fq(0)] * (upto**n)
    for k in iter_multi(dimsU):
        out[idx_from_multi(k, stridesU)] = flatL[idx_from_multi(k, stridesL)]
    return out, dimsU

# ===== N polynomials product via per-variable small-length NTT =====
def ntt_product_dense_multilinear(polys, num_vars=n):
    """
    polys: list of multilinear dense polynomials in R
    Returns: product polynomial computed via per-axis NTT/DFT.
    """
    N = len(polys)
    if N == 0:
        return R.one()
    # choose transform length
    L = pick_length_L(N)  # smallest odd L >= N+1 that divides p-1
    w = primitive_Lth_root(L)
    F, Finv = precompute_dft_tables(L, w)

    # transform each polynomial to L^n grid in the frequency domain
    spectra = []
    for P in polys:
        coeffs2, dims2 = poly_to_tensor2(P, num_vars)
        coeffsL, dimsL = pad_tensor(coeffs2, dims2, L)
        nd_dft_inplace(coeffsL, dimsL, F)
        spectra.append(coeffsL)

    # pointwise multiply spectra
    total = L**num_vars
    Y = [Fq(1)] * total
    for S in spectra:
        for i in range(total):
            Y[i] *= S[i]

    # inverse transform
    nd_idft_inplace(Y, dimsL, Finv)

    # crop to (N+1)^n (no wraparound because L >= N+1)
    coeffs_out, dims_out = crop_tensor(Y, dimsL, N+1)

    # assemble polynomial
    P_out = tensor2_to_poly(coeffs_out, dims_out, R, X)
    return P_out

# ===== Your generator (unchanged), just re-used for fair comparison =====
def generate_dense_multilinear_polynomial(R, num_vars=9):
    variables = R.gens()
    poly = R.zero()
    for combination in product([0, 1], repeat=num_vars):
        coeff = Fq.random_element()
        term = coeff
        for i, bit in enumerate(combination):
            if bit == 1:
                term *= variables[i]
            else:
                term *= (1 - variables[i])
        poly += term
    return poly

# ===== Validation harness =====
def cross_validate(N, num_vars=9, eval_checks=5):
    print(f"\n=== Cross-validate with N={N}, n={num_vars} ===")
    # Generate polys
    t0 = time.time()
    polys = [generate_dense_multilinear_polynomial(R, num_vars) for _ in range(N)]
    t1 = time.time()
    print(f"Generated {N} polys in {t1-t0:.3f}s")

    # Naive product
    t2 = time.time()
    P_naive = polys[0]
    for i in range(1, N):
        P_naive *= polys[i]
    t3 = time.time()
    print(f"Naive product time: {t3-t2:.3f}s")

    # NTT product
    t4 = time.time()
    P_ntt = ntt_product_dense_multilinear(polys, num_vars)
    t5 = time.time()
    print(f"NTT product time: {t5-t4:.3f}s")

    # Exact equality for small outputs; randomized for large
    # (N=2 has 3^n=19,683 terms -> exact is fine; N=3 has 262,144 -> can be heavy)
    terms_est = (N+1)**num_vars
    if terms_est <= 30000:
        equal = (P_naive - P_ntt).is_zero()
        print(f"Exact coefficient check: {'OK' if equal else 'MISMATCH'}")
    else:
        equal = True
        for r in range(eval_checks):
            point = {X[i]: Fq.random_element() for i in range(num_vars)}
            v1 = P_naive(**point)
            v2 = P_ntt(**point)
            if v1 != v2:
                equal = False
                break
        print(f"Randomized eval check ({eval_checks} pts): {'OK' if equal else 'MISMATCH'}")

    return {
        "gen_time": t1-t0, "naive_time": t3-t2, "ntt_time": t5-t4,
        "equal": equal, "terms_est": terms_est
    }

res2 = cross_validate(2, num_vars=n)
res3 = cross_validate(3, num_vars=n)
# You can try N=4; product has 5^9 terms (~1.95M).
# On pure Python loops this will work but will be slower without Cython:
# res4 = cross_validate(4, num_vars=n)



=== Cross-validate with N=2, n=9 ===
Generated 2 polys in 0.350s
Naive product time: 0.386s
NTT product time: 61.289s
Exact coefficient check: OK

=== Cross-validate with N=3, n=9 ===
Generated 3 polys in 0.871s
Naive product time: 21.327s


KeyboardInterrupt: 